In [1]:
import sys
sys.path.append(r'../dotNet/libs')

import clr
clr.AddReference("Microsoft.ML.Probabilistic")
clr.AddReference("Microsoft.ML.Probabilistic.Compiler")
clr.AddReference("Microsoft.ML.Probabilistic.Learners")

from System import Array, Double, Type

In [2]:
from Microsoft.ML.Probabilistic.Models import Variable, VariableArray, Range, InferenceEngine
from Microsoft.ML.Probabilistic.Algorithms import VariationalMessagePassing
from Microsoft.ML.Probabilistic.Distributions import Gaussian, Gamma

### Example 1: coin toss

This is the implementation of https://dotnet.github.io/infer/userguide/a%20simple%20example.html

In [3]:
firstCoin = Variable.Bernoulli(Double(0.5))
secondCoin = Variable.Bernoulli(Double(0.5))
bothHeads = firstCoin.op_BitwiseAnd(firstCoin, secondCoin)

In [4]:
engine = InferenceEngine()
# This should give 'Bernoulli(0.25)' since probability
# that a coin toss will give heads for both coins equals to 0.5 * 0.5 == 0.25
print(f'Probability distribution both coins are heads: {engine.Infer(bothHeads)}');
# Now we observe a coin toss with observed value for both heads == false
bothHeads.ObservedValue = False;  
# Probability that the first coin is head == 1/3, since
# there are only 3 events possible without both coins giving heads
print(f'Probability distribution over firstCoin: {engine.Infer(firstCoin)}');

Probability distribution both coins are heads: Bernoulli(0.25)
Probability distribution over firstCoin: Bernoulli(0.3333)


### Example 2: Inference on Gaussian with priors

This is the implementation of https://dotnet.github.io/infer/userguide/Running%20inference.html

**Note**: as mentioned in the README, we can't call `Variable.Observed` as in the C# code. Another issue encountered is the usage of the `[]` indexer, which doesn't seem to be correctly resolved by python.net. The code below is a workaround that initializes the `VariableArray` differently.  

In [5]:
mean_point = 0.0
precision_point = 1.0
# define the priors for the mean and precision of the Gaussian
mean = Variable.GaussianFromMeanAndVariance(mean_point, 100.0)
precision = Variable.GammaFromShapeAndScale(precision_point, 1.0)

# define observed values as drawn from a Gaussian(mean, precision)
i = Range(4)
data = Variable.Array[Double](i)
data.set_Item(i, Variable.GaussianFromMeanAndPrecision(mean, precision).ForEach(i))
data.set_ObservedValue(Array[Double]([11.0, 5.0, 8.0, 9.0]))

In [6]:
# run inference
engine = InferenceEngine(VariationalMessagePassing()) 

# get the posterior distributions
posteriorMean = engine.Infer[Gaussian](mean)  
posteriorPrecision = engine.Infer[Gamma](precision)  

# This should show the following output:
# mean = Gaussian(8.165, 1.026)
# precision = Gamma(3, 0.08038)[mean=0.2411]
print(f'mean = {posteriorMean}')  
print(f'precision = {posteriorPrecision}')

mean = Gaussian(8.165, 1.026)
precision = Gamma(3, 0.08038)[mean=0.2411]


If we sample observations from the mean prior Gaussian with increasing number of samples, we should see the posterior marginals converge to the distributions centered around the mean values with decreasing variance.

In [7]:
def observations_Gaussian(mean, precision,
                          mean_dist, precision_dist, nb_samples):
    i = Range(nb_samples)
    data = Variable.Array[Double](i)
    data.set_Item(i, 
        Variable.GaussianFromMeanAndPrecision(mean_dist, precision_dist).ForEach(i))
    data.set_ObservedValue(Array[Double](
        [Gaussian.Sample(mean, precision) for _ in range(nb_samples)]))
    
    return data

In [8]:
engine = InferenceEngine(VariationalMessagePassing()) 

for count in range(15):
    nb_samples = 5 * 2**count 
    
    # reset the priors
    mean = Variable.GaussianFromMeanAndVariance(mean_point, 100.0)
    precision = Variable.GammaFromShapeAndScale(precision_point, 1.0)
    
    data = observations_Gaussian(mean_point, precision_point,
                                 mean, precision, nb_samples)

    print(f'number of samples: {nb_samples}')
    print(f'mean = {engine.Infer[Gaussian](mean)}')  
    print(f'precision = {engine.Infer[Gamma](precision)}')

number of samples: 5
mean = Gaussian(-0.1316, 0.1651)
precision = Gamma(3.5, 0.3456)[mean=1.21]
number of samples: 10
mean = Gaussian(-0.08821, 0.04277)
precision = Gamma(6, 0.3895)[mean=2.337]
number of samples: 20
mean = Gaussian(0.2009, 0.08888)
precision = Gamma(11, 0.05109)[mean=0.562]
number of samples: 40
mean = Gaussian(0.05247, 0.02275)
precision = Gamma(21, 0.05232)[mean=1.099]
number of samples: 80
mean = Gaussian(-0.05073, 0.01625)
precision = Gamma(41, 0.01876)[mean=0.7691]
number of samples: 160
mean = Gaussian(-0.05915, 0.006072)
precision = Gamma(81, 0.01271)[mean=1.029]
number of samples: 320
mean = Gaussian(0.1047, 0.002724)
precision = Gamma(161, 0.007125)[mean=1.147]
number of samples: 640
mean = Gaussian(-0.01588, 0.001604)
precision = Gamma(321, 0.003034)[mean=0.974]
number of samples: 1280
mean = Gaussian(-0.002367, 0.0008159)
precision = Gamma(641, 0.001494)[mean=0.9575]
number of samples: 2560
mean = Gaussian(0.01041, 0.0003936)
precision = Gamma(1281, 0.000774

If we sample observations from a Gaussian with different parameters than the mean values of the priors, the posterior marginals will now converge to distributions that are centered around different values.

In [9]:
for count in range(15):
    nb_samples = 5 * 2**count 
    
    # reset the priors
    mean = Variable.GaussianFromMeanAndVariance(mean_point, 100.0)
    precision = Variable.GammaFromShapeAndScale(precision_point, 1.0)
    
    data = observations_Gaussian(-2.0, 4.0,
                                 mean, precision, nb_samples)

    print(f'number of samples: {nb_samples}')
    print(f'mean = {engine.Infer[Gaussian](mean)}')  
    print(f'precision = {engine.Infer[Gamma](precision)}')

number of samples: 5
mean = Gaussian(-1.825, 0.08638)
precision = Gamma(3.5, 0.661)[mean=2.313]
number of samples: 10
mean = Gaussian(-1.95, 0.02001)
precision = Gamma(6, 0.8326)[mean=4.995]
number of samples: 20
mean = Gaussian(-1.968, 0.01896)
precision = Gamma(11, 0.2397)[mean=2.637]
number of samples: 40
mean = Gaussian(-1.988, 0.006959)
precision = Gamma(21, 0.171)[mean=3.592]
number of samples: 80
mean = Gaussian(-2.035, 0.003839)
precision = Gamma(41, 0.07942)[mean=3.256]
number of samples: 160
mean = Gaussian(-2.024, 0.00157)
precision = Gamma(81, 0.04916)[mean=3.982]
number of samples: 320
mean = Gaussian(-1.97, 0.0008799)
precision = Gamma(161, 0.02206)[mean=3.552]
number of samples: 640
mean = Gaussian(-2.003, 0.0004106)
precision = Gamma(321, 0.01185)[mean=3.805]
number of samples: 1280
mean = Gaussian(-2.005, 0.0002031)
precision = Gamma(641, 0.006001)[mean=3.846]
number of samples: 2560
mean = Gaussian(-2, 9.658e-05)
precision = Gamma(1281, 0.003157)[mean=4.045]
number of